<a href="https://colab.research.google.com/github/doguma/domet/blob/main/01_DataCollection_TheMet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install selenium
! pip install webcolors
! pip install --upgrade google-cloud-bigquery

! pip install validators

! apt-get update
! apt install chromium-chromedriver
! cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [4]:
import os, time, sys
import cv2, keras, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import Counter

from google.oauth2 import service_account
from google.cloud import bigquery

import re
import webcolors
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from PIL import Image, ImageColor

from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
def start_chromedriver():
    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    
    return driver

In [ ]:
# Hex to RGB Conversion
# Source: https://gist.github.com/kb22/f17e59a79d4fcca02188c23cca932be5#file-rgb2hex-py

def hex2name(c):
    h_color = '#{:02x}{:02x}{:02x}'.format(int(c[0]), int(c[1]), int(c[2]))

    rms_lst = []
    for img_clr, img_hex in webcolors.CSS3_NAMES_TO_HEX.items():
        cur_clr = webcolors.hex_to_rgb(img_hex)
        rmse = np.sqrt(mean_squared_error(c, cur_clr))
        rms_lst.append(rmse)

    closest_color = rms_lst.index(min(rms_lst))

    nm = list(webcolors.CSS3_NAMES_TO_HEX.items())[closest_color][0]

    return nm

In [ ]:
# connecting to google cloud platform

! gcloud iam service-accounts create dopoem
! gcloud projects add-iam-policy-binding dopoem --member="serviceAccount:dopoem@dopoem.iam.gserviceaccount.com" --role="roles/owner"
! gcloud auth login

! export GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads"
! export GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads/service-account-file.json"

In [ ]:
credentials = "dopoem-f5aa6b994716.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials

In [ ]:
# Google Cloud's client to pull data

client = bigquery.Client()

In [ ]:
# Querying images and vision_api_data from "bigquery-public-data"

QUERY4 = ("""
SELECT * FROM (
  SELECT *
  FROM bigquery-public-data.the_met.images img
  LEFT JOIN (
    SELECT 
        *
    FROM bigquery-public-data.the_met.vision_api_data
  ) vis
  ON img.object_id = vis.object_id
)
LIMIT 500
""")

In [ ]:
def remove_space(string):
    return re.sub(r'\n', '', string)

In [ ]:
hello, bye, good = [], [], []

In [ ]:
bye.append('so')

In [ ]:
bye

['so']

In [ ]:
% time

import pandas as pd
import validators
import numpy as np
import urllib
import urllib.request
import cv2
from google.colab.patches import cv2_imshow
import io
from io import BytesIO
from numpy import asarray
import requests
import PIL

met_images, met_id, type1, type2, type3, type4, color1, color2, color3, title, text = [], [], [], [], [], [], [], [], [], [], []

query_job4 = client.query(QUERY4)
rows4 = query_job4.result()


def url_to_image(link):
  response = requests.get(link)
  # get image links that only respond with status code of 200
  # ignore the 'not found' pages
  if response.status_code == 200:
    image_bytes = io.BytesIO(response.content)
    img = PIL.Image.open(image_bytes)
    return img
  else:
    return None


for i in rows4:

      image = url_to_image(i[3])
      if image != None:
        met_id.append(i[0])
        img_numpy = asarray(image)
        met_images.append(img_numpy)

      driver = start_chromedriver()
      driver.get('https://www.metmuseum.org/art/collection/search/' + str(i[0]))
      html = driver.page_source

      soup = BeautifulSoup(html, 'html.parser')
 
      title.append(soup.find('span', 'artwork__title__inset').text)
      text.append(soup.find('div', 'artwork__intro__desc').text)

      driver.quit()

      # try and except due to missing values
      try:
        type1.append(i[12][0]['description'])
      except:
        type1.append(None)
      try:
        type2.append(i[12][1]['description'])
      except:
        type2.append(None)
      try:
        type3.append(i[12][2]['description'])
      except:
        type3.append(None)
      try:
        type4.append(i[12][3]['description'])
      except:
        type4.append(None)

      # I should use functions more next time..
      color1.append(hex2name((i[15]['dominantColors']['colors'][0]['color']['blue'],\
              i[15]['dominantColors']['colors'][0]['color']['green'],\
              i[15]['dominantColors']['colors'][0]['color']['red'])))

      color2.append(hex2name((i[15]['dominantColors']['colors'][1]['color']['blue'],\
              i[15]['dominantColors']['colors'][1]['color']['green'],\
              i[15]['dominantColors']['colors'][1]['color']['red'])))
      
      color3.append(hex2name((i[15]['dominantColors']['colors'][2]['color']['blue'],\
              i[15]['dominantColors']['colors'][2]['color']['green'],\
              i[15]['dominantColors']['colors'][2]['color']['red'])))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 8.34 µs


In [ ]:
query_job4 = client.query(QUERY4)
rows4 = query_job4.result()

total_metid = []

for i in rows4:
  total_metid.append(i[0])

In [ ]:
met_collection = pd.DataFrame()
met_collection['id'] = met_id
met_collection['images'] = met_images

In [ ]:
# saving id and images data as a csv file
met_collection.to_csv('met_collec_images.csv', index=False)  
!cp met_collec_images.csv /content/gdrive/My\ Drive

In [ ]:
met_collec_data = pd.DataFrame()
met_collec_data['id'] = total_metid
met_collec_data['type1'] = type1
met_collec_data['type2'] = type2
met_collec_data['type3'] = type3
met_collec_data['type4'] = type4
met_collec_data['color1'] = color1
met_collec_data['color2'] = color2
met_collec_data['color3'] = color3
met_collec_data['title'] = title
met_collec_data['text'] = text

In [ ]:
met_collec_data['text'] = met_collec_data['text'].apply(remove_space)
met_collec_data['title'] = met_collec_data['title'].apply(remove_space)

In [ ]:
met_collec_data.head()

,id,type1,type2,type3,type4,color1,color2,color3,title,text
0,437065,ancient history,art,abbot,temple,steelblue,black,darkslateblue,Saint Paul,Converted to Christianity on the road to Damas...
1,438427,painting,modern art,art,mythology,darkslategray,black,lightsteelblue,The Lamentation,"In this astonishing painting, Christ has been ..."
2,212207,font,drawing,sketch,art,gray,darkslategray,snow,Candlestick,
3,209192,sketch,drawing,figure drawing,arm,darkgray,gray,dimgray,Panel (part of a set),
4,189834,black and white,sketch,drawing,silver,silver,darkgray,gray,Pair of shoes,


In [ ]:
# saving id and image details data as a csv file
met_collec_data.to_csv('met_collec_data.csv', index=False) 
!cp met_collec_data.csv /content/gdrive/My\ Drive

In [ ]:
# merge two dataframes by id
total_met_collec = met_collection.merge(met_collec_data, on='id', how='left')
total_met_collec.head()

,id,images,type1,type2,type3,type4,color1,color2,color3,title,text
0,209192,"[[[199, 199, 199], [199, 199, 199], [200, 200,...",sketch,drawing,figure drawing,arm,darkgray,gray,dimgray,Panel (part of a set),
1,189834,"[[[255, 255, 255], [254, 254, 254], [253, 253,...",black and white,sketch,drawing,silver,silver,darkgray,gray,Pair of shoes,
2,191926,"[[[212, 212, 212], [212, 212, 212], [211, 211,...",black and white,vase,lighting,dishware,darkgray,silver,gray,Sugar bowl with cover,
3,188030,"[[[200, 200, 200], [199, 199, 199], [200, 200,...",money,coin,currency,silver,lightslategray,gray,silver,In Honor of the 50th Birthday of the Poet and ...,
4,188502,"[[[231, 231, 231], [232, 232, 232], [231, 231,...",money,currency,coin,silver,dimgray,dimgray,lightslategray,Die,


In [ ]:
# saving combined dataframe - total met data as a csv file
total_met_collec.to_csv('total_met_data.csv', index=False) 
!cp total_met_data.csv /content/gdrive/My\ Drive